In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Step 1: Read CSV file
data = pd.read_csv('/Users/shreyupadhyay/Downloads/gooddata_proton_pluse_180_pluse_lenvsfreq19 - gooddata_proton_pluse_180_pluse_lenvsfreq19.csv')

# Step 2: Data Preprocessing (if needed)

# Step 3: Define the sharp bell curve function (e.g., Gaussian)
def gaussian(x, amplitude, mean, stddev):
    return amplitude * np.exp(-((x - mean) / stddev) ** 2)

# Step 4: Fit the curve to the data
x_data = data[:,0]
y_data = data[:,1]

initial_guess = [1, np.mean(x_data), 1]  # Initial guess for amplitude, mean, and standard deviation
params, covariance = curve_fit(gaussian, x_data, y_data, p0=initial_guess)

# Step 5: Plotting
plt.figure(figsize=(8, 6))
plt.scatter(x_data, y_data, label='Data')
plt.plot(x_data, gaussian(x_data, *params), color='red', label='Fitted Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Sharp Bell Curve Fitting')
plt.legend()
plt.grid(True)
plt.show()


InvalidIndexError: (slice(None, None, None), 0)

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Step 1: Read data from CSV file
data = np.loadtxt('/Users/shreyupadhyay/Downloads/gooddata_proton_pluse_180_pluse_lenvsfreq19 - gooddata_proton_pluse_180_pluse_lenvsfreq19.csv')

x_data = data[:, 0]
y_data = data[:, 1]

# Step 2: Define the bell curve function
def bell_curve(x, a, b, c, d):
    return a * np.exp(-(x - b)**2 / (2 * c**2)) + d

# Step 3: Perform curve fitting
initial_guess = [1, 0, 1, 0]  # Initial guess for parameters
params, cov = curve_fit(bell_curve, x_data, y_data, initial_guess)

# Step 4: Plot the data and the fitted curve
plt.scatter(x_data, y_data, label='Data')
plt.plot(x_data, bell_curve(x_data, *params), color='red', label='Fitted Curve')

plt.xlabel('X')
plt.ylabel('Y')
plt.title('Fitted Sharp Bell Curve')
plt.legend()
plt.grid(True)
plt.show()

print("Fitted parameters:")
print("a =", params[0])
print("b =", params[1])
print("c =", params[2])
print("d =", params[3])


ValueError: could not convert string '-3.01652,-0.367254,-0.00479829' to float64 at row 0, column 1.

In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, minimize
from scipy.interpolate import interp1d
from qutip import *

# Program 1)
# Parameters
Delta = 0
Omega = 2 * np.pi
Gamma_values = [0.1 * Omega, Omega, 3 * Omega]
csv_file_path = '/Users/shreyupadhyay/Downloads/gooddata_proton_pluse_180_pluse_lenvsfreq19 - gooddata_proton_pluse_180_pluse_lenvsfreq19.csv'

# Create a list to store the data
# Function to calculate total photon scattering rate
def photon_scattering_rate(Delta, Gamma, s):
    H = -0.5 * Delta * sigmaz() + 0.5 * Omega * (sigmax() + sigmay())
    c_ops = [np.sqrt(Gamma) * sigmam()]

    # Steady-state solver
    rho_ss = steadystate(H, c_ops)
    Pe_ss = expect(sigmap() * sigmam(), rho_ss)

    # Calculate total photon scattering rate
    rate = Gamma * Pe_ss * (1 + s) / (1 + s + Delta**2 / Gamma**2)
    return rate

# Time array for plotting (can be adjusted as needed)
Delta_values = np.linspace(-5, 5, 1000)

# Function to calculate rates for given Gamma and s
def calculate_rates(Gamma, s):
    return [photon_scattering_rate(D, Gamma, s) for D in Delta_values]

# Program 2)
# Load data from the second graph
timestamps = []
values = []

with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        timestamp, value = map(float, row)
        timestamps.append(timestamp)
        values.append(value)

# Define the bell-shaped curve function (Gaussian)
def bell_curve(timestamp, a, b, c, d):
    return a * np.exp(-(timestamp - b)**2 / (2 * c**2)) + d

# Perform curve fit
initial_guess = [max(values), np.mean(timestamps), np.std(timestamps), min(values)]
params, covariance = curve_fit(bell_curve, timestamps, values, p0=initial_guess)

# Plot the original data
plt.plot(timestamps, values, marker='o', linestyle='-', label='Original Data')

# Plot the fitted curve
fitted_values = bell_curve(np.array(timestamps), *params)
plt.plot(timestamps, fitted_values, linestyle='--', color='red', label='Fitted Curve')

# Invert y-axis
plt.gca().invert_yaxis()

# Plot settings
plt.title('Curve Fit')
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

# Optimize parameters for program 1)
# Function to minimize (difference between the two programs)
def minimize_difference(params):
    Gamma, s = params
    rates_program1 = calculate_rates(Gamma, s)

    # Interpolate fitted values to match the shape of rates_program1
    interp_fitted_values = interp1d(timestamps, fitted_values, kind='linear', fill_value='extrapolate')
    interpolated_fitted_values = interp_fitted_values(Delta_values)

    return np.sum((np.array(rates_program1) - interpolated_fitted_values)**2)

# Initial guess for gamma and s
initial_guess_params = [0.1 * Omega, 2 * Omega**2 / (0.1 * Omega)**2]

# Optimize parameters
result = minimize(minimize_difference, initial_guess_params, method='Nelder-Mead')

# Extract optimized parameters
optimal_gamma, optimal_s = result.x

# Print optimized parameters
print(f"Optimal Gamma: {optimal_gamma}")
print(f"Optimal s: {optimal_s}")

# Plot the optimized curve for program 1)
rates_optimized = calculate_rates(optimal_gamma, optimal_s)

plt.plot(Delta_values, rates_optimized, label=f'Optimized (Γ = {optimal_gamma/Omega}, s = {optimal_s})')
plt.xlabel('Detuning (Δ)')
plt.ylabel('Total Photon Scattering Rate')
plt.legend()
plt.show()


ValueError: too many values to unpack (expected 2)

In [9]:
pip install seaborn

  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/83/11/00d3c3dfc25ad54e731d91449895a79e4bf2384dc3ac01809010ba88f6d5/seaborn-0.13.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 2.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

from scipy import stats
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

In [12]:
folder_path = '/Users/shreyupadhyay/Documents/lab1_test.csv'
df_data = pd.read_csv(folder_path, sep="\t", header=None)

#Covert Panda dataframe to numpy
df_numpy=df_data.to_numpy(dtype ='float32')


FileNotFoundError: [Errno 2] No such file or directory: '/Users/shreyupadhyay/Documents/lab1_test.csv'